## TensorFlow
- 선형 대수 심볼 컴파일러이다. Theano에 비해 분산 처리 기능이 강화되어 있다.

- 1. 심볼 변수 정의
- 2. 심볼 관계 정의
- 3. 세션 정의
- 4. 세션 사용
- 세션은 Theano의 함수와 유사한 역할을 하며 실제 심볼 변수의 관계를 분석하고 값을 계산해준다.

TensorFlow는 Variable 명령으로 정의하며 Theano와 달리 각 심볼 변수의 형태를 스칼라, 벡터, 행렬 등으로 명시적으로 정의하지 않는 대신 초기값을 이용해서 형태를 지정한다. 심볼의 이름은 명시적으로 주지 않아도 된다.

In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'tensorflow'

In [7]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

In [8]:
type(x), type(y)

(tensorflow.python.ops.variables.Variable,
 tensorflow.python.ops.variables.Variable)

In [9]:
z = x + y

In [10]:
type(z)

tensorflow.python.framework.ops.Tensor

In [11]:
u = tf.log(z)

In [12]:
type(u)

tensorflow.python.framework.ops.Tensor

TensorFlow의 세션은 Theano의 함수와 유사한 역할을 하며 실제로 계산 그래프를 생성하고 값을 계산하기 위한 환경을 제공한다. Theano의 함수와 달리 세션 생성과 실행 시작, 종료를 다음과 같은 방법으로 명시해야 한다.
- 세션 생성 : Session 객체 생성
- 세션 사용 : run 메서드. 그래프의 값을 실제로 계산하고 결과를 반환한다.
- 세션 종료 : close 메서드. with 문으로 대체하기도 한다.
- 그리고 각 변수의 초기값을 대입하기 위한 특별한 그래프인 global_variables_initializer를 생성하고 실행해야 한다.

In [13]:
sess = tf.Session()
init = tf.global_variables_initializer()
# make speacial graph by allocating initial value
sess.run(init)
print(sess.run(z))
print(sess.run(u))
sess.close()

3.0
1.0986123


TensorFlow 도 심볼릭 연산에 의한 미분 계산이 가능하다. 도함수를 만들 때는 gradients 명령을 사용한다.

In [14]:
f = x**2
fx = tf.gradients(f,[x])
# In theano, gradients method was `grad` 
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(f))
    print(sess.run(fx))

1.0
[2.0]


TensorFlow에는 Theano와 달리 최적화를 위한 `GradientDescentOptimizer` 등의 클래스가 미리 구현되어 있으므로 사용자가 구현할 필요가 없다.

## Tensorboard 용 로그 생성
- 텐서플로 처리 과정에서 로그 파일을 만들면 추후 텐서보드 등의 프로그램에서 읽어서 웹 화면에 나타낼 뿐이다. 따라서 계산 코드에서 로그 생성을 위한 코드를 추가해 주어야 한다.
- 로그 코드는 다음과 같은 순서로 추가한다.
    1. `summary.histogram` 명령으로 개별 변수 `기록용 summary 생성`
    2. `summary.merge` 명령으로 개별 변수 기록용 `summmary를 하나의 것으로 합침`
    3. 세션 생성후 `tf.summary.FileWriter` 명령으로 `기록용 writer 객체 생성`(위 예제에서는 writer). 이 때 기록할 디렉토리 설정
    4. `sess.run` 명령으로 summary 객체(위 예제에서는 merged)를 보내면 기록 실시
    5. 기록용 write 객체의 `add_summary` 명령으로 summary를 실제 파일에 쓰기.

In [4]:
from sklearn.datasets import load_iris
iris = load_iris()

idx = np.in1d(iris.target, [0, 2])
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] - 1.0)[:, np.newaxis] # y=1 또는 y=-1. 2차원임을 명시

In [5]:
np.random.seed(0)
w = tf.Variable(1e-3 * np.random.randn(2, 1))
b = tf.Variable(1e-3 * np.random.randn(1)[0])
d = tf.tanh(tf.matmul(X_data, w) + b)

zero = np.zeros(1, dtype=np.float64)[0] # float64 자료형의 0 값 생성 
cost = tf.reduce_sum(tf.maximum(zero, tf.multiply(-y_data, d)))
optimizer = tf.train.GradientDescentOptimizer(1e-7)# factory class(step size)
train = optimizer.minimize(cost)# train is for updating the comp

sess = tf.Session()

w_summary = tf.summary.histogram("w", w)
b_summary = tf.summary.histogram("b", b)
d_summary = tf.summary.histogram("d", d)
cost_summary = tf.summary.histogram("cost", cost)
merged = tf.summary.merge([w_summary, b_summary, d_summary, cost_summary])

init = tf.global_variables_initializer()
sess.run(init)

writer = tf.summary.FileWriter("/home/hskimim/logs", sess.graph)

for i in range(100):
    sess.run(train)
    summary = sess.run(merged)
    writer.add_summary(summary, i)
    if i % 10 == 0:
        print("{:4d}: ".format(i), sess.run(cost))

   0:  0.5494091733308988
  10:  0.4550718147888182
  20:  0.3607278431087564
  30:  0.2663786243415365
  40:  0.17202552496054824
  50:  0.07766991175203047
  60:  0.0009515019028440592
  70:  0.0003168280919044532
  80:  0.0002921058755255818
  90:  0.0002922898747369561


In [6]:
y_pred = sess.run(tf.sign(d))

In [7]:
sess.close()

In [8]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_data, y_pred)

array([[49,  1],
       [ 1, 49]])

In [50]:
from sklearn.datasets import load_iris
iris = load_iris()

idx = np.in1d(iris.target, [0, 2])
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] - 1.0)[:, np.newaxis] # y=1 또는 y=-1. 2차원임을 명시
np.random.seed(0)
w = tf.Variable(1e-3 * np.random.randn(2, 1))
b = tf.Variable(1e-3 * np.random.randn(1)[0])
d = tf.tanh(tf.matmul(X_data, w) + b)

zero = np.zeros(1, dtype=np.float64)[0] # float64 자료형의 0 값 생성 
cost = tf.reduce_sum(tf.maximum(zero, tf.multiply(-y_data, d)))
optimizer = tf.train.GradientDescentOptimizer(1e-7)
train = optimizer.minimize(cost)

sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)


for i in range(100):
    sess.run(train)
    if i % 10 == 0:
        print("{:4d}: ".format(i), sess.run(cost))

   0:  0.5494091733308988
  10:  0.4550718147888182
  20:  0.3607278431087564
  30:  0.2663786243415365
  40:  0.17202552496054824
  50:  0.07766991175203047
  60:  0.0009515019028440592
  70:  0.0003168280919044532
  80:  0.0002921058755255818
  90:  0.0002922898747369561


In [51]:
y_pred = sess.run(tf.sign(d))
sess.close()
from sklearn.metrics import confusion_matrix

confusion_matrix(y_data, y_pred)

array([[49,  1],
       [ 1, 49]])

#### 연습 문제 1
- 위 퍼셉트론 코드를 다음과 같이 고쳐라.
    - y값은 0과 1이라는 값만 가진다.
    - 활성화 함수는 로지스틱 함수를 사용한다.
    - 코스트 함수는 다음과 같은 오차 제곱 합수를 사용한다.

In [27]:
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()

idx = np.in1d(iris.target, [0, 2])
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] - 1.0)[:, np.newaxis] 
# y=1 또는 y=-1. 2차원임을 명시

In [44]:
np.random.seed(0)
w = tf.Variable(1e-2 * np.random.randn(2, 1))
b = tf.Variable(1e-2 * np.random.randn(1)[0])
d = 1 / (1 + tf.exp(-(tf.matmul(X_data, w) + b)))

zero = np.zeros(1, dtype=np.float64)[0] # float64 자료형의 0 값 생성 
cost = tf.reduce_sum(tf.square(y_data - d))
optimizer = tf.train.GradientDescentOptimizer(1e-2)
train = optimizer.minimize(cost)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [45]:
for i in range(100000):
    sess.run(train)
    if i % 5000 == 0:
        print("{:4d}: ".format(i), sess.run(cost))


   0:  23.089692786456325
5000:  0.6179743797251298
10000:  0.4429841429727435
15000:  0.36432122030317815
20000:  0.3215152805682526
25000:  0.2938979513044885
30000:  0.2739215908492397
35000:  0.2583384270042387
40000:  0.24553876193040775
45000:  0.23463454999187516
50000:  0.22509475512844496
55000:  0.21658158118657522
60000:  0.2088694346873602
65000:  0.2018015918607085
70000:  0.19526552747200795
75000:  0.18917812986948235
80000:  0.1834764584977726
85000:  0.17811176035553816
90000:  0.1730454816219592
95000:  0.1682465433230773


In [46]:
y_pred = sess.run(tf.round(d))

In [47]:
sess.close()

In [48]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_data, y_pred)

array([[50,  0],
       [ 0, 50]])